# Описание проекта

Подготовьте прототип модели машинного обучения для «Цифры». Компания разрабатывает решения для эффективной работы промышленных предприятий.

Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. В вашем распоряжении данные с параметрами добычи и очистки.

Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

**Задача:**

* Подготовить данные;
* Провести исследовательский анализ данных;
* Построить и обучить модель.
* Чтобы выполнить проект, обращайтесь к библиотекам pandas, matplotlib и sklearn. Вам поможет их документация.

## Описание данных

*Технологический процесс*

* Rougher feed — исходное сырье
* Rougher additions (или reagent additions) — флотационные реагенты: Xanthate, * Sulphate, Depressant
* Xanthate **— ксантогенат (промотер, или активатор флотации);
* Sulphate — сульфат (на данном производстве сульфид натрия);
* Depressant — депрессант (силикат натрия).
* Rougher process (англ. «грубый процесс») — флотация
* Rougher tails — отвальные хвосты
* Float banks — флотационная установка
* Cleaner process — очистка
* Rougher Au — черновой концентрат золота
* Final Au — финальный концентрат золота

*Параметры этапов*
* air amount — объём воздуха
* fluid levels — уровень жидкости
* feed size — размер гранул сырья
* feed rate — скорость подачи

**Наименование признаков**

Наименование признаков должно быть такое:\
**[этап].[тип_параметра].[название_параметра]**\
Пример: rougher.input.feed_ag

*Возможные значения для блока [этап]:* 

* rougher — флотация
* primary_cleaner — первичная очистка
* secondary_cleaner — вторичная очистка
* final — финальные характеристики

*Возможные значения для блока [тип_параметра]:*

* input — параметры сырья
* output — параметры продукта
* state — параметры, характеризующие текущее состояние этапа
* calculation — расчётные характеристики


# Подготовка и предобработка данных

In [1]:
#common
import numpy as np
import pandas as pd 
import IPython
from IPython.display import display
import warnings
warnings.simplefilter('ignore')
import scipy.stats as st
import re

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

#ml
from sklearn.metrics import SCORERS, make_scorer
from sklearn.metrics import mean_squared_error,  r2_score, roc_auc_score, roc_curve, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, LabelEncoder, Binarizer
from sklearn.preprocessing import QuantileTransformer, PolynomialFeatures
from sklearn.utils import shuffle, resample
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA

#classifiers

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [2]:
full = pd.read_csv('gold_recovery_full.csv')
train = pd.read_csv('gold_recovery_train.csv')
test = pd.read_csv('gold_recovery_test.csv')


In [3]:
train.drop(['Unnamed: 0'], axis=1, inplace=True)
test.drop(['Unnamed: 0'], axis=1, inplace=True)
full.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
display(train.head())
display(test.head())
display(full.head())

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,...,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,10.128295,7.25,0.988759,8.547551,10.389648,19.529297,34.174427,14.936526,2.534912,7.476074,2.106679,1549.775757,-498.912140,1551.434204,-516.403442,1549.873901,-498.666595,1554.367432,-493.428131,41885.707031,3481.779053,3520.337158,2.838687,6.100378,2.284912,523.546326,55.486599,36.808594,6.486150,...,19.793808,87.107763,5.008018,0.508728,19.154297,1.170244,999.706909,-404.066986,1603.011353,-434.715027,1602.375000,-442.204468,1598.937256,-451.294128,1404.472046,-455.462982,1416.354980,-451.939636,14.500184,4.694824,8.764648,2.606185,25.853109,-498.526489,23.893660,-501.406281,23.961798,-495.262817,21.940409,-499.340973,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,10.296251,7.25,1.002663,8.558743,10.497069,19.369147,34.118526,16.250511,3.049646,6.733907,2.353017,1576.166671,-500.904965,1575.950626,-499.865889,1575.994189,-499.315107,1574.479259,-498.931665,42050.861809,3498.370979,3489.981857,2.858998,6.161113,2.266033,525.290581,57.278666,35.753385,6.478583,...,20.050975,86.843261,

,date,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-09-01 00:59:59,210.800909,14.993118,8.080000,1.005021,1398.981301,-500.225577,1399.144926,-499.919735,1400.102998,-500.704369,1399.001695,-499.485144,13.128569,5.636710,489.793655,62.710452,42.022440,12.084103,16.922877,6.153479,16.867518,6.150709,1001.849317,-350.301265,1249.738612,-399.107531,1249.750520,-399.396730,1198.287855,-399.488594,999.472250,-399.530962,949.566189,-398.180807,24.938200,-500.490963,14.947607,-500.013837,20.018166,-450.395995,13.987058,-449.831513,12.023554,-497.795834,8.016656,-501.289139,7.946562,-432.317850,4.872511,-500.037437,26.705889,-499.709414
1,2016-09-01 01:59:59,215.392455,14.987471,8.080000,0.990469,1398.777912,-500.057435,1398.055362,-499.778182,1396.151033,-499.240168,1399.508091,-500.416324,13.036280,5.526065,490.104534,61.961163,41.187903,11.918977,17.002532,5.999043,16.996063,6.002315,998.690785,-350.425704,1248.395457,-399.946025,1249.514451,-399.630810,1200.506459,-399.942434,1000.002114,-399.491528,950.200009,-405.787514,24.923466,-499.813963,14.930873,-500.764452,19.988503,-450.114024,14.092981,-450.059163,12.058140,-498.695773,8.130979,-499.634209,7.958270,-525.839648,4.878850,-500.162375,25.019940,-499.819438
2,2016-09-01 02:59:59,215.259946,12.884934,7.786667,0.996043,1398.493666,-500.868360,1398.860436,-499.764529,1398.075709,-502.151509,1399.497088,-499.715479,13.138412,5.426561,489.617969,66.903807,42.546294,12.091332,16.992520,5.850632,16.982096,5.853594,998.516720,-349.783538,1247.440748,-400.264330,1248.207204,-401.074405,1199.770026,-400.790582,999.925519,-399.236999,950.320481,-400.864340,24.908732,-500.304035,14.996511,-500.993821,20.039689,-450.263164,14.077881,-449.660997,11.962366,-498.767484,8.096893,-500.827423,8.071056,-500.801673,4.905125,-499.828510,24.994862,-500.622559
3,2016-09-01 03:59:59,215.336236,12.006805,7.640000,0.863514,1399.618111,-498.863574,1397.440120,-499.211024,1400.129303,-498.355873,1401.065165,-501.038738,12.400507,5.114758,476.618130,59.865919,41.060195,12.181597,16.532144,5.801935,16.515277,5.803657,1000.276604,-350.172597,1251.322675,-398.655289,1250.493702,-399.746728,1199.398691,-397.50

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,...,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,10.128295,7.25,0.988759,8.547551,10.389648,19.529297,34.174427,14.936526,2.534912,7.476074,2.106679,1549.775757,-498.912140,1551.434204,-516.403442,1549.873901,-498.666595,1554.367432,-493.428131,41885.707031,3481.779053,3520.337158,2.838687,6.100378,2.284912,523.546326,55.486599,36.808594,6.486150,...,19.793808,87.107763,5.008018,0.508728,19.154297,1.170244,999.706909,-404.066986,1603.011353,-434.715027,1602.375000,-442.204468,1598.937256,-451.294128,1404.472046,-455.462982,1416.354980,-451.939636,14.500184,4.694824,8.764648,2.606185,25.853109,-498.526489,23.893660,-501.406281,23.961798,-495.262817,21.940409,-499.340973,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,10.296251,7.25,1.002663,8.558743,10.497069,19.369147,34.118526,16.250511,3.049646,6.733907,2.353017,1576.166671,-500.904965,1575.950626,-499.865889,1575.994189,-499.315107,1574.479259,-498.931665,42050.861809,3498.370979,3489.981857,2.858998,6.161113,2.266033,525.290581,57.278666,35.753385,6.478583,...,20.050975,86.843261,

In [5]:
#Посмотрим размеры выборок
print('Размер обучающей выборки: {} '.format(train.shape))
print('Размер тестовой выборки: {} '.format(test.shape))
print('Размер общей выборки: {} '.format(full.shape))

Размер обучающей выборки: (16860, 87) 
Размер тестовой выборки: (5856, 53) 
Размер общей выборки: (22716, 87) 


In [6]:
#Изучим наши выборки
display(train.info())
display(test.info())
display(full.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.tail_sol                               16715 non-null  float64


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   date                                        5856 non-null   object 
 1   primary_cleaner.input.sulfate               5554 non-null   float64
 2   primary_cleaner.input.depressant            5572 non-null   float64
 3   primary_cleaner.input.feed_size             5856 non-null   float64
 4   primary_cleaner.input.xanthate              5690 non-null   float64
 5   primary_cleaner.state.floatbank8_a_air      5840 non-null   float64
 6   primary_cleaner.state.floatbank8_a_level    5840 non-null   float64
 7   primary_cleaner.state.floatbank8_b_air      5840 non-null   float64
 8   primary_cleaner.state.floatbank8_b_level    5840 non-null   float64
 9   primary_cleaner.state.floatbank8_c_air      5840 non-null   float64
 10  primary_clea

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                22716 non-null  object 
 1   final.output.concentrate_ag                         22627 non-null  float64
 2   final.output.concentrate_pb                         22629 non-null  float64
 3   final.output.concentrate_sol                        22331 non-null  float64
 4   final.output.concentrate_au                         22630 non-null  float64
 5   final.output.recovery                               20753 non-null  float64
 6   final.output.tail_ag                                22633 non-null  float64
 7   final.output.tail_pb                                22516 non-null  float64
 8   final.output.tail_sol                               22445 non-null  float64


None

In [7]:
#Проверим на наличие дубликатов
print('Тестовая выборка, количество дубликатов: ',test.duplicated().sum())
print('Обучающая выборка, количество дубликатов: ',train.duplicated().sum())
print('Full Dataset, количество дубликатов: ',full.duplicated().sum())

Тестовая выборка, количество дубликатов:  0
Обучающая выборка, количество дубликатов:  0
Full Dataset, количество дубликатов:  0


**Вывод**

* 87, 87 и 53 столбца
* 22 тысяч строк. Присутствуют пропущенные значения. Дата в формате 'object'. В сетах full и train присутствуют данные о содержании серебра (Ag), зола (?) (Sol), свинец (Pb) и золото (Au)
* Возможно, ненужные элементы можно удалить
* Вероятно, каждому сету стоит присвоить индексы = дате - и потом через новые индексы производить необходимые преобразования сетов



## 1.1 Проверим, что эффективность обогащения рассчитана правильно. Вычислим ее на обучающей выборке для признака "rougher.output.recovery". Найдем MAE между нашими расчетами и значениями признака. Опишем выводы.**

In [8]:
def recovery(c,f,t):

#Расчитаем эффективность с помощью формулы recovery данной в инструкции.

    prerecovery = (c*(f-t)) / (f*(c-t))
    recovery = prerecovery * 100

# Заполним слишком маленькие и слишком большие значения с помощью np.nan
    recovery[recovery<0] = np.nan
    recovery[recovery>100] = np.nan
    
    return recovery

In [9]:
c = train['rougher.output.concentrate_au']
f = train['rougher.input.feed_au']
t = train['rougher.output.tail_au']

recovery_list = recovery(c,f,t)

In [10]:
#Посчитаем MAE, но прежде заполним NAN значения на 0
mae = mean_absolute_error(train['rougher.output.recovery'].fillna(0),recovery_list.fillna(0))
print('Средняя абсолютная ошибка:', mae)

Средняя абсолютная ошибка: 8.188520377116694e-15



Значение МАЕ крайне мало (отклонение рассчетных данных от представленных может быть не более 8.2e-15), следовательно, представленные данные рассчитаны верно.

In [11]:
#Посчитаем количество пустых значений в наших расчетах и признаках
print('Пустых значений в recovery:', recovery_list.isna().sum())
print('Пустых значений в train[recovery]:' ,train['rougher.output.recovery'].isna().sum())

Пустых значений в recovery: 2573
Пустых значений в train[recovery]: 2573


In [12]:
# Потеря данных в процентах
train_clear = train.dropna()
print('Потеря данных составит', 
      100 - int((len(train_clear) * 100 / len(train))), '%')

Потеря данных составит 35 %


## 1.2 Анализ недоступных признаков тестовой выборки

In [13]:
train_set = train.columns
test_set = test.columns

In [14]:
difference = (list(set(train_set)-set(test_set)))
print('Количество недостающих признаков в тестовом сете = ', len(difference))
print()
print('Признаки, отсутствующие в тестовом сете:')
print()
print(np.sort(difference))

Количество недостающих признаков в тестовом сете =  34

Признаки, отсутствующие в тестовом сете:

['final.output.concentrate_ag' 'final.output.concentrate_au'
 'final.output.concentrate_pb' 'final.output.concentrate_sol'
 'final.output.recovery' 'final.output.tail_ag' 'final.output.tail_au'
 'final.output.tail_pb' 'final.output.tail_sol'
 'primary_cleaner.output.concentrate_ag'
 'primary_cleaner.output.concentrate_au'
 'primary_cleaner.output.concentrate_pb'
 'primary_cleaner.output.concentrate_sol' 'primary_cleaner.output.tail_ag'
 'primary_cleaner.output.tail_au' 'primary_cleaner.output.tail_pb'
 'primary_cleaner.output.tail_sol' 'rougher.calculation.au_pb_ratio'
 'rougher.calculation.floatbank10_sulfate_to_au_feed'
 'rougher.calculation.floatbank11_sulfate_to_au_feed'
 'rougher.calculation.sulfate_to_au_concentrate'
 'rougher.output.concentrate_ag' 'rougher.output.concentrate_au'
 'rougher.output.concentrate_pb' 'rougher.output.concentrate_sol'
 'rougher.output.recovery' 'rougher.ou

* В тестовой выборке отсутствуют признаки final.output, primary_cleaner.output, rougher.output, secondary_cleaner.output.
* Данные относятся параметрам продукта, параметрам характеризующие текущее состояние этапа и расчетные характеристики.


## 1.3 Проведем предобработку данных

**перевод даты**


In [15]:
train['date_time'] = pd.to_datetime(train['date'], format='%Y-%m-%dT%H:%M:%S')
train.drop(columns=['date'], inplace=True)
test['date_time'] = pd.to_datetime(test['date'], format='%Y-%m-%dT%H:%M:%S')
test.drop(columns=['date'], inplace=True)
full['date_time'] = pd.to_datetime(full['date'], format='%Y-%m-%dT%H:%M:%S')
full.drop(columns=['date'], inplace=True)

**Заполнение пропусков**\
Проведем анализ пропущенных значений во всех сетах.

In [16]:
full.isna().sum().sort_values(ascending=False)

rougher.output.recovery                     3119
rougher.output.tail_ag                      2737
rougher.output.tail_au                      2736
rougher.output.tail_sol                     2736
rougher.input.floatbank11_xanthate          2257
                                            ... 
primary_cleaner.state.floatbank8_c_level      43
primary_cleaner.state.floatbank8_d_level      43
primary_cleaner.state.floatbank8_b_level      43
primary_cleaner.input.feed_size                0
date_time                                      0
Length: 87, dtype: int64

In [17]:
train.isna().sum().sort_values(ascending=False)

rougher.output.recovery                          2573
rougher.output.tail_ag                           2250
rougher.output.tail_au                           2249
rougher.output.tail_sol                          2249
secondary_cleaner.output.tail_sol                1986
                                                 ... 
rougher.calculation.sulfate_to_au_concentrate      27
primary_cleaner.state.floatbank8_d_level           27
primary_cleaner.state.floatbank8_b_level           27
primary_cleaner.input.feed_size                     0
date_time                                           0
Length: 87, dtype: int64

In [18]:
test.isna().sum().sort_values(ascending=False).head(5)

rougher.input.floatbank11_xanthate    353
primary_cleaner.input.sulfate         302
primary_cleaner.input.depressant      284
rougher.input.floatbank10_sulfate     257
primary_cleaner.input.xanthate        166
dtype: int64

Больше всего пропущенных значений в одном из целевых столбцов 'rougher.output.recovery'

Удалим наблюдения с отсутствующими целевыми признаками, так как восстанавливать их по "усредненным" значениям, как начинающий data-scientist считаю неправильным.

In [19]:
full = full.dropna(axis='index', how='any', subset=['rougher.output.recovery', 'final.output.recovery'])
train = train.dropna(axis='index', how='any', subset=['rougher.output.recovery', 'final.output.recovery'])

Остальные пропуски, заполним ближайшими (верхними) значениями по столбцам, поскольку из условий проекта следует, что "Соседние по времени параметры часто похожи."

In [20]:
full = full.fillna(method='ffill')
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [21]:
print(full.isna().sum().unique()) 
print(train.isna().sum().unique())
print(test.isna().sum().unique())

[0]
[0]
[0]


In [22]:
print('В обучающей выборке : {} строк {} признаков'.format(train.shape[0],train.shape[1]))
print('В тестовой выборке: {} строк {} признаков'.format(test.shape[0], test.shape[1]))

В обучающей выборке : 14149 строк 87 признаков
В тестовой выборке: 5856 строк 53 признаков


**Отбросим из сета train все фичи, которые не встречаются в сете test, чтобы уровнять количество признаков для построения моделей.**

* удалим целевые признаки из переменной difference, в которой у нас сохранены отличающиеся столбцы train и test сетов.
* Удалим обновленный список столбцов из сета train.

In [23]:
unwanted = {'rougher.output.recovery','final.output.recovery'}

difference_ = [e for e in difference if e not in unwanted]

train_upd = train.drop(difference_, axis=1)

train_upd.shape[1], test.shape[1]

(55, 53)

Разница в 2 столбца - это отсуствие таргета в сете test.

Добавим таргеты из full в test, соотвтетсвие установим по индексу.

Финальная проверка соответсвия столбцов train_upd и test_upd, подготовленных для дальнейшей обработки

In [24]:
train_cols = train_upd.columns.tolist()
test_cols = test_upd.columns.tolist()

list(set(train_cols) - set(test_cols)), list(set(test_cols) - set(train_cols))

NameError: ignored

Оба списки пустые - в двух сетах одинаковые столбцы.

Удалим пропущенные значения в таргетах в сете test_upd, которые попали сюда при добавлении столбцов с таргетом и full.

In [ ]:
test_upd.isna().sum().sort_values(ascending=False).head(2)

In [ ]:
test_upd = test_upd.dropna(axis='index', how='any', subset=['rougher.output.recovery', 'final.output.recovery'])

In [ ]:
test_upd.isna().sum().sort_values(ascending=False).head()

Соединение данных в тестовом сете произведено верно.
Сортировать порядок столбцов не имеет смысла. Можно сбросить все индексы, но пока оставим, как есть, возможно, где-то в дальнейшем, они еще пригодятся.
В выборках train_upd и test_upd по 55 одинаковых столбцов.



## Вывод

На данном шаге произведено знакомство с данными, и выпоплены первые шаги проекта. За время работы установлено следующее:

* В предложенных данных ~ 30% пропущенных значений;
* Целевыми признаками данных являются столбцы 'rougher.output.recovery' и 'final.output.recovery';
* Рассчетные значения столбца 'rougher.output.recovery' в сете recovery_train демонстрируют значение средней абсолютной ошибки, в сравнении с представленными значениями, равной 9.4e-15. Можно утверждать, что представленные данные посчитаны корректно.
* В процессе предобработки дата была преведена в читаемый вид, были удалены строки с пропущенным значенем целевых признаков во всех трех сетах, пропущенные значения фич заполнены методом "ближаших верхних соседей" - "ffill".
* Из первоначального сета recovery_train были удалены фичи, которых нет в recovery_test. В recovery_test были добавлены целевые признаки из recovery_full.

В целом, за время предобработы, было удалено ~16% изначальных данных, за счет дропа неизвестных целевых признаков.

full готов для исследования признаков (пункт 2 проекта)
train_upd и test_upd подготовленны для построения моделей (пункт 3 проекта)

